In [25]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
#filling missing values
train["Age"] = train["Age"].fillna(30)
train["Embarked"].fillna("S", inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [19]:
#Features
train_x = train.drop(["PassengerId","Name","Ticket","Cabin","Survived"], axis=1)
#label
train_y = train["Survived"]
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [26]:
#transform non-numerical labels to numerical labels
encoder = LabelEncoder()
for col in ('Sex','Embarked'):
    train_x['Sex'] = encoder.fit_transform(train_x['Sex'])
    train_x['Embarked'] = encoder.fit_transform(train_x['Embarked'])

In [27]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, 
                                                  test_size=0.2, 
                                                  random_state=1)

In [28]:
d_train = xgb.DMatrix(train_x, train_y)
d_test = xgb.DMatrix(test_x, test_y)

In [34]:
param = {
    'booster':'gbtree',
    'eta':0.3, #prevents overfitting
    'max_depth':3, #maximum depth of tree
    'objective':'binary:hinge',
    'rate_drop':0.1,
    'n_estimators':500
}

steṕs = 20

In [35]:
model = xgb.train(param,d_train,steṕs)

[21:06:45] WARNING: /workspace/src/learner.cc:480: 
Parameters: { n_estimators, rate_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [36]:
#predictions
preds = model.predict(d_test)
#print accuracy
print(np.round(accuracy_score(test_y,preds)*100, 2),'%')

82.52 %


In [37]:
#XGBClassifier scikit-learn API 
model = xgb.XGBClassifier(max_depth = 3, learning_rate=0.1)
model.fit(train_x, train_y)
model.score(test_x,test_y) 

0.8111888111888111